<a href="https://colab.research.google.com/github/pchen35/Python101/blob/master/QA_simpletransformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# 安裝simpletransformers，用於調用Transformer模型
!pip install simpletransformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.5/250.5 KB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 88.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 39.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 60.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 KB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 113.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 99.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 108.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 24.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [2]:
# 下載SQuAD資料集
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json

--2023-03-15 04:05:48--  https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.111.153, 185.199.110.153, 185.199.108.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.111.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42123633 (40M) [application/json]
Saving to: ‘train-v2.0.json.1’

train-v2.0.json.1   100%[===================>]  40.17M  --.-KB/s    in 0.09s   

2023-03-15 04:05:48 (446 MB/s) - ‘train-v2.0.json.1’ saved [42123633/42123633]



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [3]:
!pip install tensorboardX

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 KB 9.8 MB/s eta 0:00:00


In [5]:
# 載入資料與訓練模型 40M Training time 約8 min
import json
from simpletransformers.question_answering import QuestionAnsweringModel

with open('.//sample_data//train-v2.0.json', 'r') as f:
    train_data = json.load(f)

train_data = [item for topic in train_data['data'] for item in topic['paragraphs'] ]

train_data = train_data[:500]

train_args = {
    'learning_rate': 3e-5,
    'num_train_epochs': 2,
    'max_seq_length': 256,
    'doc_stride': 128,
    'overwrite_output_dir': True,
    'reprocess_input_data': False,
    'train_batch_size': 12,
    'fp16': False,
}
model = QuestionAnsweringModel('distilbert', 'distilbert-base-cased', args=train_args)
model.train_model(train_data)

Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertForQuestionAnswering: ['vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on

add example index and unique id: 100%|██████████| 3840/3840 [00:00<00:00, 365533.28it/s]


Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Running Epoch 0 of 2:   0%|          | 0/382 [00:00<?, ?it/s]

Running Epoch 1 of 2:   0%|          | 0/382 [00:00<?, ?it/s]

(764, 2.445466433598109)

In [7]:
# 模型評估(evaluation data)
with open('.//sample_data//dev-v2.0.json', 'r') as f:
    eval_data = json.load(f)
eval_data = [item for topic in eval_data['data'] for item in topic['paragraphs'] ]
eval_data = eval_data[:100]
result, text = model.eval_model(eval_data)

Running Evaluation:   0%|          | 0/99 [00:00<?, ?it/s]

In [8]:
result #評估結果

{'correct': 106,
 'similar': 474,
 'incorrect': 157,
 'eval_loss': -5.777271073273938}

In [9]:
# 使用模型進行推論（這部分可以自己寫例子）
to_predict = [
    {
        "context": "Other legislation followed, including the Migratory Bird Conservation Act of 1929, a 1937 treaty prohibiting the hunting of right and gray whales,\
            and the Bald Eagle Protection Act of 1940. These later laws had a low cost to society—the species were relatively rare—and little opposition was raised",
        "qas": [{"question": "What was the name of the 1937 treaty?", "id": "0"}], #正確答案是prohibiting the hunting of right and gray whales
    }
]

print(model.predict(to_predict))

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 11275.01it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

([{'id': '0', 'answer': ['Bald Eagle Protection', 'Bald Eagle Protection Act of 1940', '1937 treaty prohibiting the hunting of right and gray whales', 'Migratory Bird Conservation Act of 1929, a 1937 treaty prohibiting the hunting of right and gray whales', 'prohibiting the hunting of right and gray whales', '1937 treaty prohibiting the hunting of right and gray whales, and the Bald Eagle Protection', 'the Bald Eagle Protection', 'Migratory Bird Conservation Act of 1929, a 1937 treaty prohibiting the hunting of right and gray whales, and the Bald Eagle Protection', 'right and gray whales', 'hunting of right and gray whales', 'prohibiting the hunting of right and gray whales, and the Bald Eagle Protection', 'Bald Eagle', 'the hunting of right and gray whales', 'Bald Eagle Protection Act', 'the Migratory Bird Conservation Act of 1929, a 1937 treaty prohibiting the hunting of right and gray whales', 'right and gray whales, and the Bald Eagle Protection', 'hunting of right and gray whales,

In [10]:
# 使用模型進行簡易問答
to_predict = [
    {
        "context": "A woman is sitting under the tree.",
        "qas": [{"question": "where is the worman?", "id": "0"}], #正確答案是prohibiting the hunting of right and gray whales
    }
]

print(model.predict(to_predict))

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 2424.45it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

([{'id': '0', 'answer': ['A woman', 'woman', 'A woman is sitting under the tree', '', 'tree', 'woman is sitting under the tree', 'under the tree', 'A woman is sitting under the tree.', 'tree.', 'woman is sitting under the tree.', 'the tree', 'A woman is sitting under', 'sitting under the tree', 'A woman is sitting', 'A', 'woman is sitting under', 'under the tree.', 'woman is sitting', 'the tree.']}], [{'id': '0', 'probability': [0.17986106384306327, 0.1213558988014788, 0.11647162624590622, 0.11335253340841159, 0.09417474977831061, 0.07858576273225455, 0.0415384477804718, 0.039214825016011465, 0.03170769098463515, 0.026459035849544295, 0.024558566659039674, 0.0225829026821411, 0.019798287291936775, 0.01703416733105003, 0.0162658352205113, 0.015237141346660923, 0.013985577549237535, 0.011493297340874646, 0.008268622369408566]}])


In [11]:
from google.colab import drive
drive.mount('/content/drive')
# 4/1AX4XfWjuMtLDP-PQOsTtfaxUubv-vwSRnClRaez0Vz2zcNuehh35Jlc9sHc
# 4/1AX4XfWgiRrlMvOPbtWOHDRu-qB-UI8DJKjliPrOPQr9q7yAs3tFTAN1b1rM

MessageError: ignored

In [ ]:
!ls -la /content/drive/MyDrive

total 32693
-r-------- 1 root root   994426 Mar 23  2020  Anaconda安裝說明手冊.docx
drwx------ 2 root root     4096 Sep 26  2018 'Colab Notebooks'
-r-------- 1 root root 32477591 Mar 23  2020  Python從入門到進階_20200324.pptx
-rw------- 1 root root      151 Sep  2 03:18  wiki_sentences_v2.gsheet


In [ ]:
!ls -la /content/drive/Shareddrives

total 4
drwx------ 27 root root 4096 Feb  4  2018 中華電信研究院


In [ ]:
!curl -X POST "https://qanswer-core1.univ-st-etienne.fr/api/gerbil?kb=dbpedia&lang=en&query=mayor%20of%20rome" -H "accept: application/json"

{"questions":[{"question":{"answers":"{\n  \"head\" : {\n    \"vars\" : [\n      \"o1\"\n    ]\n  },\n  \"results\" : {\n    \"bindings\" : [\n      {\n        \"o1\" : {\n          \"type\" : \"uri\",\n          \"value\" : \"http://dbpedia.org/resource/City_Council_of_Rome\"\n        }\n      },\n      {\n        \"o1\" : {\n          \"xml:lang\" : \"en\",\n          \"type\" : \"literal\",\n          \"value\" : \"Virginia Raggi\"\n        }\n      }\n    ]\n  }\n}","language":[{"SPARQL":"SELECT DISTINCT ?o1 WHERE { \t <http://dbpedia.org/resource/Rome>  <http://dbpedia.org/property/leaderName>  ?o1 .  }  LIMIT 1000","confidence":0.98}]}}]}